In [1]:
!ls

sample_data


In [1]:
!git clone https://github.com/Samsung-IT-Academy/stepik-dl-nlp.git
import sys; sys.path.append('/content/stepik-dl-nlp')

!pip install pyconll
!pip install spacy-udpipe 

fatal: destination path 'stepik-dl-nlp' already exists and is not an empty directory.


In [2]:
!wget -O /content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
!wget -O /content/stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2019-12-16 08:11:29--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81043533 (77M) [text/plain]
Saving to: ‘/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu’

/content/stepik-dl- 100%[===================>]  77.29M   273MB/s    in 0.3s    

2019-12-16 08:11:30 (273 MB/s) - ‘/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu’ saved [81043533/81043533]

--2019-12-16 08:11:31--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubu

In [0]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import classification_report

import numpy as np

import pyconll

import torch
from torch import nn
from torch.nn import functional as F
from torch.utils.data import TensorDataset

import dlnlputils
from dlnlputils.data import tokenize_corpus, build_vocabulary, \
    character_tokenize, pos_corpus_to_tensor, POSTagger
from dlnlputils.pipeline import train_eval_loop, predict_with_model, init_random_seed
import collections
import re

init_random_seed()

def build_vocabulary(tokenized_texts, max_size=1000000, max_doc_freq=0.8, min_count=5, pad_word=None):
    word_counts = collections.defaultdict(int)
    doc_n = 0

    # посчитать количество документов, в которых употребляется каждое слово
    # а также общее количество документов
    for txt in tokenized_texts:
        doc_n += 1
        unique_text_tokens = set(txt)
        for token in unique_text_tokens:
            word_counts[token] += 1

    # убрать слишком редкие и слишком частые слова
    word_counts = {word: cnt for word, cnt in word_counts.items()
                   if cnt >= min_count and cnt / doc_n <= max_doc_freq}

    # отсортировать слова по убыванию частоты
    sorted_word_counts = sorted(word_counts.items(),
                                reverse=True,
                                key=lambda pair: pair[1])

    # добавим несуществующее слово с индексом 0 для удобства пакетной обработки
    if pad_word is not None:
        #sorted_word_counts = [(pad_word, 0)] + sorted_word_counts
        sorted_word_counts = sorted_word_counts + [(pad_word, len(sorted_word_counts)+1)]

    # если у нас по прежнему слишком много слов, оставить только max_size самых частотных
    if len(word_counts) > max_size:
        sorted_word_counts = sorted_word_counts[:max_size]

    # нумеруем слова
    word2id = {word: i for i, (word, _) in enumerate(sorted_word_counts)}

    # нормируем частоты слов
    word2freq = np.array([cnt / doc_n for _, cnt in sorted_word_counts], dtype='float32')

    return word2id, word2freq


In [4]:
full_train = pyconll.load_from_file('/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-train.conllu')
full_test = pyconll.load_from_file('/content/stepik-dl-nlp/datasets/ru_syntagrus-ud-dev.conllu')

MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Наибольшая длина предложения', MAX_SENT_LEN)
print('Наибольшая длина токена', MAX_ORIG_TOKEN_LEN)

all_train_texts = [' '.join(token.form for token in sent) for sent in full_train]
print('\n'.join(all_train_texts[:10]))

train_char_tokenized = tokenize_corpus(all_train_texts, tokenizer=character_tokenize)
char_vocab, word_doc_freq = build_vocabulary(train_char_tokenized, max_doc_freq=1.0, min_count=5, pad_word='<PAD>')
print("Количество уникальных символов", len(char_vocab))
print(list(char_vocab.items())[:10])

UNIQUE_TAGS = ['<NOTAG>'] + sorted({token.upos for sent in full_train for token in sent if token.upos})
label2id = {label: i for i, label in enumerate(UNIQUE_TAGS)}

train_inputs, train_labels = pos_corpus_to_tensor(full_train, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
train_dataset = TensorDataset(train_inputs, train_labels)

test_inputs, test_labels = pos_corpus_to_tensor(full_test, char_vocab, label2id, MAX_SENT_LEN, MAX_ORIG_TOKEN_LEN)
test_dataset = TensorDataset(test_inputs, test_labels)

Наибольшая длина предложения 205
Наибольшая длина токена 47
Анкета .
Начальник областного управления связи Семен Еремеевич был человек простой , приходил на работу всегда вовремя , здоровался с секретаршей за руку и иногда даже писал в стенгазету заметки под псевдонимом " Муха " .
В приемной его с утра ожидали посетители , - кое-кто с важными делами , а кое-кто и с такими , которые легко можно было решить в нижестоящих инстанциях , не затрудняя Семена Еремеевича .
Однако стиль работы Семена Еремеевича заключался в том , чтобы принимать всех желающих и лично вникать в дело .
Приемная была обставлена просто , но по-деловому .
У двери стоял стол секретарши , на столе - пишущая машинка с широкой кареткой .
В углу висел репродуктор и играло радио для развлечения ожидающих и еще для того , чтобы заглушать голос начальника , доносившийся из кабинета , так как , бесспорно , среди посетителей могли находиться и случайные люди .
Кабинет отличался скромностью , присущей Семену Еремеевичу .
В глуб

In [35]:
def get_class_weights(labels):
  count_labels = np.unique(labels, return_counts= True)[1]
  return count_labels/count_labels[1:].sum()
  #exclude 0 <PAD> weight
  # return count_labels[1:]/count_labels[1:].sum()

def get_label_with_min_weight(labels, class_weights):
  labels = np.unique(labels)
  label_weight = {l:class_weights[l] for l in labels}
  return min(label_weight.items(), key=lambda item: item[1])

class_weights = get_class_weights(train_labels)  
print(class_weights)
#get_label_with_min_weight(train_labels[349], class_weights)  

def get_label_weight(labels, class_weights, n_weighted = 2):
  labels = np.unique(labels)
  label_weight = {l:class_weights[l] for l in labels}
  
  weights = sorted(label_weight.items(), key=lambda item: item[1])
  return sum(map(lambda x: x[1], weights[:n_weighted]))

# array([9.13011861e-01, 8.55302407e-03, 8.19067301e-03, 4.40707234e-03,
#        7.51683593e-04, 3.04111076e-03, 2.19529183e-03, 7.79464508e-06,
#        2.14349742e-02, 1.37365630e-03, 2.66327033e-03, 3.84116112e-03,
#        3.23787558e-03, 1.57880536e-02, 1.62078652e-03, 8.39423316e-05,
#        9.76029468e-03, 3.74742552e-05])

[1.04958201e+01 9.83240262e-02 9.41585035e-02 5.06629109e-02
 8.64121937e-03 3.49600622e-02 2.52366800e-02 8.96058377e-05
 2.46412607e-01 1.57913057e-02 3.06164767e-02 4.41572973e-02
 3.72220352e-02 1.81496624e-01 1.86322703e-02 9.64985945e-04
 1.12202592e-01 4.30797297e-04]


In [33]:


# weights = get_label_weight(train_labels[349],class_weights)
# sum(weights, ke)

class_weights = get_class_weights(train_labels)  
# weight_labels = [get_label_weight(label, class_weights)   for label in train_labels]
# weight_labels
class_weights


array([9.13011861e-01, 8.55302407e-03, 8.19067301e-03, 4.40707234e-03,
       7.51683593e-04, 3.04111076e-03, 2.19529183e-03, 7.79464508e-06,
       2.14349742e-02, 1.37365630e-03, 2.66327033e-03, 3.84116112e-03,
       3.23787558e-03, 1.57880536e-02, 1.62078652e-03, 8.39423316e-05,
       9.76029468e-03, 3.74742552e-05])

In [0]:
import torch
import torch.utils.data
import torchvision

def get_class_weights(labels):
  count_labels = np.unique(labels, return_counts= True)[1]
  return count_labels/count_labels.sum()
  #exclude 0 <PAD> weight
  # return count_labels[1:]/count_labels[1:].sum()

def get_label_weight(labels, class_weights, n_weighted = 5):
  labels = np.unique(labels)
  label_weight = {l:class_weights[l] for l in labels}
  
  weights = sorted(label_weight.items(), key=lambda item: item[1] if item[0] != 0 else 0)
  return sum(map(lambda x: x[1], weights[:n_weighted]))
  


def get_label_with_min_weight(labels, class_weights):
  labels = np.unique(labels)
  label_weight = {l:class_weights[l] for l in labels}
  return min(label_weight.items(), key=lambda item: item[1])

class ImbalancedDatasetSampler(torch.utils.data.sampler.Sampler):
    """Samples elements randomly from a given list of indices for imbalanced dataset
    Arguments:
        indices (list, optional): a list of indices
        num_samples (int, optional): number of samples to draw
    """

    def __init__(self, dataset):
                
        # if indices is not provided, 
        # all elements in the dataset will be considered
        self.indices = np.arange(dataset.shape[0])
            
        # if num_samples is not provided, 
        # draw `len(indices)` samples in each iteration
        self.num_samples = len(self.indices)
            
        class_weights = get_class_weights(dataset)
        # weight_labels = [get_label_with_min_weight(label, class_weights)   for label in dataset]
        weight_labels = [get_label_weight(label, class_weights)   for label in dataset]

        # # weight for each sample
        self.weights = [weight_labels[idx] for idx in self.indices]
        self.weights = torch.DoubleTensor(self.weights)
                
    def __iter__(self):
        return (self.indices[i] for i in torch.multinomial(
            self.weights, self.num_samples, replacement=True))

    def __len__(self):
        return self.num_samples

In [17]:
np.unique(train_labels, return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17]),
 array([9136391,   85589,   81963,   44101,    7522,   30432,   21968,
             78,  214497,   13746,   26651,   38438,   32401,  157989,
          16219,     840,   97670,     375]))

In [10]:

# def get_class_weights(labels):
#   count_labels = np.unique(labels, return_counts= True)[1]
#   return count_labels/count_labels.sum()
#   #exclude 0 <PAD> weight
#   # return count_labels[1:]/count_labels[1:].sum()

# def get_label_weight(labels, class_weights, n_weighted = 2):
#   labels = np.unique(labels)
#   label_weight = {l:class_weights[l] for l in labels}
  
#   weights = sorted(label_weight.items(), key=lambda item: item[1])
#   return sum(map(lambda x: x[1], weights[:n_weighted]))

# num_samples = train_labels.shape[0]
# class_weights = get_class_weights(train_labels)  
# print(class_weights)
# weight_labels = [get_label_weight(label, class_weights, n_weighted = 5)   for label in train_labels]
# weights = [weight_labels[idx] for idx in np.arange(num_samples)]

# #get_label_with_min_weight(train_labels[349], class_weights)  

# #torch.utils.data.WeightedRandomSampler(weights, num_samples, replacement=True)

sampler =  ImbalancedDatasetSampler(train_labels)
#sampler = torch.utils.data.WeightedRandomSampler(weights, num_samples)
ids = [x for x in sampler]

np.unique(train_labels[ids], return_counts=True)

(array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17]),
 array([9140468,   85433,   81344,   44061,    7404,   30233,   21630,
             72,  213661,   13497,   26469,   38040,   32138,  157519,
          16095,     852,   97587,     367]))

In [25]:
array([9136391,   85589,   81963,   44101,    7522,   30432,   21968,
             78,  214497,   13746,   26651,   38438,   32401,  157989,
          16219,     840,   97670,     375]))

array([9.13011861e-01, 8.55302407e-03, 8.19067301e-03, 4.40707234e-03,
       7.51683593e-04, 3.04111076e-03, 2.19529183e-03, 7.79464508e-06,
       2.14349742e-02, 1.37365630e-03, 2.66327033e-03, 3.84116112e-03,
       3.23787558e-03, 1.57880536e-02, 1.62078652e-03, 8.39423316e-05,
       9.76029468e-03, 3.74742552e-05])

In [56]:
#np.log(1-4.30797297e-04)

-0.00043089011681421034

In [80]:
from sklearn.utils.class_weight import compute_class_weight

#min_weight_labels = [get_label_with_min_weight(l, class_weights)[0] for l in train_labels]

compute_class_weight('balanced',
      np.unique(min_weight_labels),
      min_weight_labels)



array([ 4.1452106 ,  2.60091645,  2.17298789,  0.47139601,  0.72364208,
        0.39148916, 40.14309211,  0.41089226,  0.55887067,  1.38048643,
        1.35114039,  9.41628086,  0.3805507 ,  4.51981481,  8.76688218,
       13.3810307 ])

In [0]:

#min_weight_labels = [get_label_with_min_weight(label, class_weights)   for label in train_labels]

In [26]:
weights[:2]

sum(map(lambda x: x[1], weights[:2]))

0.0038786353774956607

In [0]:
from sklearn.utils.class_weight import compute_class_weight

compute_class_weight('balanced',
      np.unique(train_labels[1]),
      train_labels[1])


In [42]:
np.array(train_labels)

array([[ 8, 13,  0, ...,  0,  0,  0],
       [ 8,  1,  8, ...,  0,  0,  0],
       [ 2,  8, 11, ...,  0,  0,  0],
       ...,
       [13,  2, 11, ...,  0,  0,  0],
       [13,  2, 11, ...,  0,  0,  0],
       [ 5, 11, 16, ...,  0,  0,  0]])

In [0]:
list(train_labels)

In [1]:
from sklearn.utils.class_weight import compute_class_weight

compute_class_weight('balanced',
      np.unique(train_labels[1]),
      train_labels[1])

#classes = {k:k for k in np.arange(class_weigts.shape[0]) }
#compute_class_weight('balanced', classes, np.array(train_labels[0]))
#classes[3]
#compute_class_weight(train_labels[0])

# indices = list(range(len(train_dataset))) 
# num_samples = len(indices) 


NameError: ignored

In [24]:
train_labels[0]
#np.arange(class_weigts.shape[0])

tensor([ 8, 13,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,  0,
         0,  0,  0,  0,  0,  0,  0])

In [0]:
class MyConv1d(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size, padding=0):
        super().__init__()
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.kernel_size = kernel_size
        self.padding = padding
        self.weight = nn.Parameter(torch.randn(in_channels * kernel_size, out_channels) / (in_channels * kernel_size),
                                   requires_grad=True)
        self.bias = nn.Parameter(torch.zeros(out_channels), requires_grad=True)
    
    def forward(self, x):
        """x - BatchSize x InChannels x SequenceLen"""

        batch_size, src_channels, sequence_len = x.shape        
        if self.padding > 0:
            pad = x.new_zeros(batch_size, src_channels, self.padding)
            x = torch.cat((pad, x, pad), dim=-1)
            sequence_len = x.shape[-1]

        chunks = []
        chunk_size = sequence_len - self.kernel_size + 1
        for offset in range(self.kernel_size):
            chunks.append(x[:, :, offset:offset + chunk_size])

        in_features = torch.cat(chunks, dim=1)  # BatchSize x InChannels * KernelSize x ChunkSize
        in_features = in_features.permute(0, 2, 1)  # BatchSize x ChunkSize x InChannels * KernelSize
        out_features = torch.bmm(in_features, self.weight.unsqueeze(0).expand(batch_size, -1, -1)) + self.bias.unsqueeze(0).unsqueeze(0)
        out_features = out_features.permute(0, 2, 1)  # BatchSize x OutChannels x ChunkSize
        return out_features

class StackedConv1d(nn.Module):
    def __init__(self, features_num, layers_n=1, kernel_size=3, conv_layer=nn.Conv1d, dropout=0.0):
        super().__init__()
        layers = []
        for layer_n in range(layers_n):
            layers.append(nn.Sequential(
                #conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),
                #conv_layer(features_num-layer_n, features_num-layer_n, kernel_size, padding=kernel_size//2, dilation=layer_n+1),                
                conv_layer(features_num, features_num, kernel_size, padding=kernel_size//2),                
                #nn.BatchNorm1d(features_num),
                nn.Dropout(dropout),                
                #nn.LeakyReLU()))
                nn.ReLU()))
        self.layers = nn.ModuleList(layers)
    
    def forward(self, x):
        """x - BatchSize x FeaturesNum x SequenceLen"""
        for layer in self.layers:
            x = x + layer(x)
        return x

class SentenceLevelPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, single_backbone_kwargs={}, context_backbone_kwargs={}):
        super().__init__()
        self.embedding_size = embedding_size
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.single_token_backbone = StackedConv1d(embedding_size, **single_backbone_kwargs)
        self.context_backbone = StackedConv1d(embedding_size, **context_backbone_kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Conv1d(embedding_size, labels_num, 1)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        char_features = self.single_token_backbone(char_embeddings)
        
        token_features_flat = self.global_pooling(char_features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize

        token_features = token_features_flat.view(batch_size, max_sent_len, self.embedding_size)  # BatchSize x MaxSentenceLen x EmbSize
        token_features = token_features.permute(0, 2, 1)  # BatchSize x EmbSize x MaxSentenceLen
        context_features = self.context_backbone(token_features)  # BatchSize x EmbSize x MaxSentenceLen

        logits = self.out(context_features)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits

class SingleTokenPOSTagger(nn.Module):
    def __init__(self, vocab_size, labels_num, embedding_size=32, **kwargs):
        super().__init__()
        self.char_embeddings = nn.Embedding(vocab_size, embedding_size, padding_idx=0)
        self.backbone = StackedConv1d(embedding_size, **kwargs)
        self.global_pooling = nn.AdaptiveMaxPool1d(1)
        self.out = nn.Linear(embedding_size, labels_num)
        self.labels_num = labels_num
    
    def forward(self, tokens):
        """tokens - BatchSize x MaxSentenceLen x MaxTokenLen"""
        batch_size, max_sent_len, max_token_len = tokens.shape
        tokens_flat = tokens.view(batch_size * max_sent_len, max_token_len)
        
        char_embeddings = self.char_embeddings(tokens_flat)  # BatchSize*MaxSentenceLen x MaxTokenLen x EmbSize
        char_embeddings = char_embeddings.permute(0, 2, 1)  # BatchSize*MaxSentenceLen x EmbSize x MaxTokenLen
        
        features = self.backbone(char_embeddings)
        
        global_features = self.global_pooling(features).squeeze(-1)  # BatchSize*MaxSentenceLen x EmbSize
        
        logits_flat = self.out(global_features)  # BatchSize*MaxSentenceLen x LabelsNum
        logits = logits_flat.view(batch_size, max_sent_len, self.labels_num)  # BatchSize x MaxSentenceLen x LabelsNum
        logits = logits.permute(0, 2, 1)  # BatchSize x LabelsNum x MaxSentenceLen
        return logits



In [6]:
sentence_level_model_my_conv = SentenceLevelPOSTagger(len(char_vocab), len(label2id), embedding_size=96,
                                                      single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=MyConv1d),
                                                      context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=MyConv1d))
                                                      # single_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d),
                                                      # context_backbone_kwargs=dict(layers_n=3, kernel_size=3, dropout=0.2, conv_layer=nn.Conv1d))

print('Количество параметров', sum(np.product(t.shape) for t in sentence_level_model_my_conv.parameters()))

Количество параметров 182610


In [0]:
from torch.utils.data import DataLoader
import copy
import datetime
import random
import traceback


def copy_data_to_device(data, device):
    if torch.is_tensor(data):
        return data.to(device)
    elif isinstance(data, (list, tuple)):
        return [copy_data_to_device(elem, device) for elem in data]
    raise ValueError('Недопустимый тип данных {}'.format(type(data)))


def train_eval_loop(model, train_dataset, val_dataset, criterion,
                    lr=1e-4, epoch_n=10, batch_size=32,
                    device=None, early_stopping_patience=10, l2_reg_alpha=0,
                    max_batches_per_epoch_train=10000,
                    max_batches_per_epoch_val=1000,
                    data_loader_ctor=DataLoader,
                    optimizer_ctor=None,
                    lr_scheduler_ctor=None,
                    shuffle_train=True,
                    dataloader_workers_n=0,
                    weights = None):
    """
    Цикл для обучения модели. После каждой эпохи качество модели оценивается по отложенной выборке.
    :param model: torch.nn.Module - обучаемая модель
    :param train_dataset: torch.utils.data.Dataset - данные для обучения
    :param val_dataset: torch.utils.data.Dataset - данные для оценки качества
    :param criterion: функция потерь для настройки модели
    :param lr: скорость обучения
    :param epoch_n: максимальное количество эпох
    :param batch_size: количество примеров, обрабатываемых моделью за одну итерацию
    :param device: cuda/cpu - устройство, на котором выполнять вычисления
    :param early_stopping_patience: наибольшее количество эпох, в течение которых допускается
        отсутствие улучшения модели, чтобы обучение продолжалось.
    :param l2_reg_alpha: коэффициент L2-регуляризации
    :param max_batches_per_epoch_train: максимальное количество итераций на одну эпоху обучения
    :param max_batches_per_epoch_val: максимальное количество итераций на одну эпоху валидации
    :param data_loader_ctor: функция для создания объекта, преобразующего датасет в батчи
        (по умолчанию torch.utils.data.DataLoader)
    :return: кортеж из двух элементов:
        - среднее значение функции потерь на валидации на лучшей эпохе
        - лучшая модель
    """
    if device is None:
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
    device = torch.device(device)
    model.to(device)
    # if weights is not None:
    #   weights = torch.tensor(weights).to(device)

    if optimizer_ctor is None:
        optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=l2_reg_alpha)
    else:
        optimizer = optimizer_ctor(model.parameters(), lr=lr)

    if lr_scheduler_ctor is not None:
        lr_scheduler = lr_scheduler_ctor(optimizer)
    else:
        lr_scheduler = None

    train_dataloader = data_loader_ctor(train_dataset, batch_size=batch_size, shuffle=shuffle_train,
                                        num_workers=dataloader_workers_n)
    val_dataloader = data_loader_ctor(val_dataset, batch_size=batch_size, shuffle=False,
                                      num_workers=dataloader_workers_n)

    best_val_loss = float('inf')
    best_epoch_i = 0
    best_model = copy.deepcopy(model)

    for epoch_i in range(epoch_n):
        try:
            epoch_start = datetime.datetime.now()
            print('Эпоха {}'.format(epoch_i))

            model.train()
            mean_train_loss = 0
            train_batches_n = 0
            for batch_i, (batch_x, batch_y) in enumerate(train_dataloader):
                if batch_i > max_batches_per_epoch_train:
                    break

                batch_x = copy_data_to_device(batch_x, device)
                batch_y = copy_data_to_device(batch_y, device)

                pred = model(batch_x)
                loss = criterion(pred, batch_y)
                # if not weights is None:
                #   loss = (loss * weights).mean()

                model.zero_grad()
                loss.backward()

                optimizer.step()

                mean_train_loss += float(loss)
                train_batches_n += 1

            mean_train_loss /= train_batches_n
            print('Эпоха: {} итераций, {:0.2f} сек'.format(train_batches_n,
                                                           (datetime.datetime.now() - epoch_start).total_seconds()))
            print('Среднее значение функции потерь на обучении', mean_train_loss)



            model.eval()
            mean_val_loss = 0
            val_batches_n = 0

            with torch.no_grad():
                for batch_i, (batch_x, batch_y) in enumerate(val_dataloader):
                    if batch_i > max_batches_per_epoch_val:
                        break

                    batch_x = copy_data_to_device(batch_x, device)
                    batch_y = copy_data_to_device(batch_y, device)

                    pred = model(batch_x)
                    loss = criterion(pred, batch_y)

                    mean_val_loss += float(loss)
                    val_batches_n += 1

            mean_val_loss /= val_batches_n
            print('Среднее значение функции потерь на валидации', mean_val_loss)

            if mean_val_loss < best_val_loss:
                best_epoch_i = epoch_i
                best_val_loss = mean_val_loss
                best_model = copy.deepcopy(model)
                print('Новая лучшая модель!')
            elif epoch_i - best_epoch_i > early_stopping_patience:
                print('Модель не улучшилась за последние {} эпох, прекращаем обучение'.format(
                    early_stopping_patience))
                break

            if lr_scheduler is not None:
                lr_scheduler.step(mean_val_loss)

            print()
        except KeyboardInterrupt:
            print('Досрочно остановлено пользователем')
            break
        except Exception as ex:
            print('Ошибка при обучении: {}\n{}'.format(ex, traceback.format_exc()))
            break

    return best_val_loss, best_model

class WeightedMultilabelCrossEntropy(nn.Module):  
    def __init__(self, weights: torch.Tensor):  
        super(WeightedMultilabelCrossEntropy, self).__init__()  
        self.cerition = F.cross_entropy #nn.BCEWithLogitsLoss(reduction='none')  
        self.weights = weights  
  
    def forward(self, outputs, targets):  
        loss = self.cerition(outputs, targets)  
        return (loss * self.weights).mean()    

def get_class_weights(labels):
  count_labels = np.unique(labels, return_counts= True)[1]
  return count_labels/count_labels.sum()
  #exclude 0 <PAD> weight
  # return count_labels[1:]/count_labels[1:].sum()


In [0]:
target = torch.tensor([[0,1,0,1,0,0]], dtype=torch.float32)
output = torch.randn(1, 6, requires_grad=True)
weights = torch.tensor([0.16, 0.16, 0.25, 0.25, 0.083, 0.083])

In [18]:


from sklearn.utils import class_weight
list_classes = np.arange(len(label2id))
y = train_labels.view(-1) #train_labels[list_classes].values
weights = class_weight.compute_sample_weight('balanced', y) #0.92
weights = torch.tensor(weights).to('cuda')
criterion = WeightedMultilabelCrossEntropy(weights) #0.93
#last <PAD> 0.93

# weights = get_class_weights(train_labels) #0.91
# weights = torch.tensor(weights).to('cuda')
# criterion = WeightedMultilabelCrossEntropy(weights) #0.93
#last <PAD> 0.92

(best_val_loss,
 best_sentence_level_model_my_conv) = train_eval_loop(sentence_level_model_my_conv,
                                                      train_dataset,
                                                      test_dataset,
                                                      F.cross_entropy, #criterion, #F.cross_entropy,
                                                      #weights=weights,
                                                      lr=5e-3,
                                                      epoch_n=10,
                                                      batch_size=64,
                                                      device='cuda',
                                                      early_stopping_patience=5,
                                                      max_batches_per_epoch_train=500,
                                                      max_batches_per_epoch_val=100,
                                                      lr_scheduler_ctor=lambda optim: torch.optim.lr_scheduler.ReduceLROnPlateau(optim, patience=2,
                                                                                                                                 factor=0.5,
                                                                                                                                 verbose=True))


Эпоха 0
Эпоха: 501 итераций, 113.93 сек
Среднее значение функции потерь на обучении 0.009429648394014188
Среднее значение функции потерь на валидации 0.008016423052545674
Новая лучшая модель!

Эпоха 1
Эпоха: 501 итераций, 113.84 сек
Среднее значение функции потерь на обучении 0.00934646191900808
Среднее значение функции потерь на валидации 0.008078449617813128

Эпоха 2
Эпоха: 501 итераций, 113.77 сек
Среднее значение функции потерь на обучении 0.00919034681143518
Среднее значение функции потерь на валидации 0.007809439683240829
Новая лучшая модель!

Эпоха 3
Эпоха: 501 итераций, 113.75 сек
Среднее значение функции потерь на обучении 0.009180242095343367
Среднее значение функции потерь на валидации 0.008346723941256209

Эпоха 4
Эпоха: 501 итераций, 113.73 сек
Среднее значение функции потерь на обучении 0.008950872356982942
Среднее значение функции потерь на валидации 0.007819625173261997

Эпоха 5
Эпоха: 501 итераций, 113.70 сек
Среднее значение функции потерь на обучении 0.00909251994927

In [19]:
train_pred = predict_with_model(best_sentence_level_model_my_conv, train_dataset)
train_loss = F.cross_entropy(torch.tensor(train_pred),
                             torch.tensor(train_labels))
print('Среднее значение функции потерь на обучении', float(train_loss))
print(classification_report(train_labels.view(-1), train_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))
print()

test_pred = predict_with_model(best_sentence_level_model_my_conv, test_dataset)
test_loss = F.cross_entropy(torch.tensor(test_pred),
                            torch.tensor(test_labels))
print('Среднее значение функции потерь на валидации', float(test_loss))
print(classification_report(test_labels.view(-1), test_pred.argmax(1).reshape(-1), target_names=UNIQUE_TAGS))

1526it [00:53, 28.50it/s]                               


Среднее значение функции потерь на обучении 0.00495589803904295


  1%|▏         | 3/205.75 [00:00<00:07, 25.61it/s]

              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   9136391
         ADJ       0.97      0.97      0.97     85589
         ADP       1.00      1.00      1.00     81963
         ADV       0.96      0.95      0.96     44101
         AUX       0.87      0.97      0.92      7522
       CCONJ       0.95      0.98      0.97     30432
         DET       0.96      0.91      0.93     21968
        INTJ       0.92      0.62      0.74        78
        NOUN       0.99      0.99      0.99    214497
         NUM       0.96      0.98      0.97     13746
        PART       0.96      0.93      0.95     26651
        PRON       0.94      0.96      0.95     38438
       PROPN       0.98      0.98      0.98     32401
       PUNCT       1.00      1.00      1.00    157989
       SCONJ       0.91      0.95      0.93     16219
         SYM       1.00      1.00      1.00       840
        VERB       0.98      0.97      0.98     97670
           X       0.98    

206it [00:07, 28.32it/s]                            


Среднее значение функции потерь на валидации 0.006752926856279373
              precision    recall  f1-score   support

     <NOTAG>       1.00      1.00      1.00   1231232
         ADJ       0.96      0.96      0.96     11222
         ADP       1.00      1.00      1.00     10585
         ADV       0.96      0.94      0.95      6165
         AUX       0.86      0.96      0.91      1106
       CCONJ       0.95      0.98      0.97      4410
         DET       0.94      0.89      0.91      3085
        INTJ       0.67      0.36      0.47        11
        NOUN       0.98      0.98      0.98     27974
         NUM       0.96      0.98      0.97      1829
        PART       0.96      0.93      0.94      3877
        PRON       0.93      0.95      0.94      5598
       PROPN       0.96      0.95      0.96      4438
       PUNCT       1.00      1.00      1.00     22694
       SCONJ       0.90      0.94      0.92      2258
         SYM       1.00      1.00      1.00        53
        VERB   